In [12]:
import pandas as pd
import pydeck as pdk
import geopandas as gpd

### 위해 지표 선정
##### 인구만명당 5대 주요 범죄 * 발생 건수(.500) - 살인, 강도, 강간, 폭력, 절도

In [14]:
crime=pd.read_csv('data/crime(5.csv')
crime.head(2)


,기간,자치구,합계,합계.1,살인,살인.1,강도,강도.1,강간강제추행,강간강제추행.1,절도,절도.1,폭력,폭력.1
0,기간,자치구,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
1,2020,합계,"92,679","66,548",141,126,138,132,"5,763","5,458","38,293","19,705","48,344","41,127"


In [3]:
pop_seoul = pd.read_csv("data/인구__가구_및_주택__읍면동_2015_2020___시군구_20162019__20220603165301.csv", encoding="cp949")
pop_seoul_2020 = pd.DataFrame(pop_seoul[["행정구역별(읍면동)", "2020"]])
column1 = ["자치구", "총인구(명)"]
pop_seoul_2020.drop([0, 1], inplace = True)
pop_seoul_2020.columns = column1
pop_seoul_2020["총인구(명)"] = pop_seoul_2020["총인구(명)"].astype(int)

In [4]:
crime_five = crime.iloc[:, [1, 2, 4, 6, 8, 10 ,12]].drop([0, 1])
crime_five.reset_index(drop=True, inplace=True)
crime_five.rename(columns={"합계": "5대범죄건수"}, inplace=True)
crime_five_total = crime_five.loc[:, ["자치구", "5대범죄건수"]]
crime_five_total["5대범죄건수"] = pd.to_numeric(crime_five_total["5대범죄건수"].str.replace(",", ""))
crime_five_total

,자치구,5대범죄건수
0,종로구,3102
1,중구,3411
2,용산구,2969
3,성동구,2362
4,광진구,3601
5,동대문구,3401
6,중랑구,3726
7,성북구,2567
8,강북구,2770
9,도봉구,2179


In [5]:
pop_10000_df = pd.merge(pop_seoul_2020, crime_five_total, how ="left", on = "자치구")
pop_10000_df["인구만명당 5대범죄건수"] = pop_10000_df["5대범죄건수"] / pop_10000_df["총인구(명)"] * 10000
pop_10000_df["위해지표"] = pop_10000_df["인구만명당 5대범죄건수"] * 0.5
pop_10000_df

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표
0,종로구,151291,3102,205.035329,102.517665
1,중구,128744,3411,264.944386,132.472193
2,용산구,225882,2969,131.440310,65.720155
3,성동구,291918,2362,80.913133,40.456567
4,광진구,353967,3601,101.732647,50.866324
5,동대문구,351057,3401,96.878854,48.439427
6,중랑구,385663,3726,96.612846,48.306423
7,성북구,438833,2567,58.496057,29.248028
8,강북구,299535,2770,92.476672,46.238336
9,도봉구,315979,2179,68.960279,34.480140


### 취약지표 선정
##### 인구만명당 집객 시설수(0.049), 인구만명당 1인가구수(0.029), 인구만명당 주점업 업체수(0.122)

In [6]:
onegagu = pd.read_csv("data/서울시1인가구_시군구별.csv", encoding="utf-8", index_col=0)
# onegagu.drop(0, inplace=True)
onegagu_df = onegagu.loc[onegagu["성별"] == "계"]
onegagu_df = onegagu_df.iloc[:, [0, 1, 34]]
onegagu_df.drop("성별", axis = 1, inplace=True)
onegagu_df.columns = ["자치구", "1인가구수"]
onegagu_df["1인가구수"] = onegagu_df["1인가구수"].astype(int)
pop_10000_df = pd.merge(pop_10000_df, onegagu_df, how ="left", on = "자치구")
pop_10000_df["인구만명당 1인가구수"] = pop_10000_df["1인가구수"] / pop_10000_df["총인구(명)"] * 10000
pop_10000_df.sort_values(by = "인구만명당 1인가구수", ascending=False)

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수
20,관악구,502641,5261,104.667148,52.333574,129233,2571.079558
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440
4,광진구,353967,3601,101.732647,50.866324,62301,1760.079329
5,동대문구,351057,3401,96.878854,48.439427,60925,1735.473157
13,마포구,365612,3688,100.871963,50.435981,63172,1727.842631
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749
17,금천구,249419,2598,104.162073,52.081036,42334,1697.304536
2,용산구,225882,2969,131.440310,65.720155,36881,1632.755155
19,동작구,392772,3200,81.472203,40.736101,64101,1632.015520
18,영등포구,403619,5217,129.255560,64.627780,64000,1585.653797


In [7]:
attraction = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-집객시설).csv", encoding="cp949")
attraction = attraction.loc[(attraction["기준_년_코드"] == 2021) & (attraction["기준_분기_코드"] == 4)]
attraction = attraction.iloc[:, 4:7]
attraction.reset_index(drop=True, inplace=True)
attraction

,상권_코드,상권_코드_명,집객시설_수
0,2130021,신설종합시장,1
1,2110343,무너미어린이공원,11
2,2111024,경찰병원역 4번(건너말공원),1
3,2130001,통인시장,2
4,2110616,양목초등학교,1
...,...,...,...
1570,2110516,정원여중,10
1571,2110565,홍익대,5
1572,2110894,관악동부센트레빌아파트(은천로35라길),1
1573,2110148,어린이대공원역 3번,1


In [8]:
commercial = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권영역).csv", encoding="cp949")
total_df = pd.merge(attraction, commercial , how = "left", on = "상권_코드")
attraction_df = total_df.iloc[:, [0, 1, 2, 9]]
attraction_df['시군구_코드']=attraction_df['시군구_코드'].replace([11680, 11740,11305,11500,11620, 11215, 11530, 11545, 11350, 11320, 11230, 11590, 11440, 11410, 11650, 11200, 11290, 11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260],['강남구','강동구','강북구','강서구','관악구','광진구', '구로구' , '금천구','노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구','서초구', '성동구','성북구', '송파구', '양천구','영등포구','용산구','은평구','종로구','중구','중랑구'])
attraction_df.rename(columns={"시군구_코드": "자치구"}, inplace = True)
attraction_df1 = attraction_df["집객시설_수"].groupby(attraction_df["자치구"]).sum()

<ipython-input-8-278ef7f7ad31>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attraction_df['시군구_코드']=attraction_df['시군구_코드'].replace([11680, 11740,11305,11500,11620, 11215, 11530, 11545, 11350, 11320, 11230, 11590, 11440, 11410, 11650, 11200, 11290, 11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260],['강남구','강동구','강북구','강서구','관악구','광진구', '구로구' , '금천구','노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구','서초구', '성동구','성북구', '송파구', '양천구','영등포구','용산구','은평구','종로구','중구','중랑구'])
c:\Users\dlagh\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [9]:
attraction_df1 = pd.DataFrame(attraction_df1)
attraction_df1.reset_index(drop=False, inplace=True)
attraction_df1

,자치구,집객시설_수
0,강남구,2831
1,강동구,1188
2,강북구,934
3,강서구,1266
4,관악구,1074
5,광진구,920
6,구로구,1175
7,금천구,887
8,노원구,922
9,도봉구,627


In [10]:
pop_10000_df = pd.merge(pop_10000_df, attraction_df1, how = "left", on = "자치구")
pop_10000_df

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수,집객시설_수
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749,1768
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440,2153
2,용산구,225882,2969,131.440310,65.720155,36881,1632.755155,982
3,성동구,291918,2362,80.913133,40.456567,42585,1458.800074,775
4,광진구,353967,3601,101.732647,50.866324,62301,1760.079329,920
5,동대문구,351057,3401,96.878854,48.439427,60925,1735.473157,1144
6,중랑구,385663,3726,96.612846,48.306423,56605,1467.732191,981
7,성북구,438833,2567,58.496057,29.248028,61510,1401.672162,1083
8,강북구,299535,2770,92.476672,46.238336,45685,1525.197389,934
9,도봉구,315979,2179,68.960279,34.480140,34884,1103.997418,627


In [11]:
pop_10000_df["인구만명당 집객 시설수"] = pop_10000_df["집객시설_수"] / pop_10000_df["총인구(명)"] * 10000

In [12]:
store = pd.read_excel("data/서울상가정보.xlsx")
store_df = store.loc[:, ["상호명", "표준산업분류명", "시군구명"]]

In [13]:
store_df.dropna(subset =["표준산업분류명"], inplace=True)
store_df1 = store_df.loc[store_df["표준산업분류명"].str.contains("주점")]
store_df1.rename(columns={"시군구명": "자치구"}, inplace=True)
store_df2 = pd.DataFrame(store_df1["자치구"].value_counts())
store_df2.reset_index(drop=False, inplace=True)
store_df2.columns = ["자치구", "주점업 업체수"]

c:\Users\dlagh\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [14]:
pop_10000_df = pd.merge(pop_10000_df, store_df2, how = "left", on = "자치구")
pop_10000_df.head(2)

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수,집객시설_수,인구만명당 집객 시설수,주점업 업체수
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749,1768,116.860884,322
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440,2153,167.231094,393


In [15]:
pop_10000_df["인구만명당 주점업 업체수"] = pop_10000_df["주점업 업체수"] / pop_10000_df["총인구(명)"] * 10000
vulnerable_indicators = (pop_10000_df["인구만명당 집객 시설수"] * 0.049) + (pop_10000_df["인구만명당 1인가구수"] * 0.029) + (pop_10000_df["인구만명당 주점업 업체수"] * 0.122)
pop_10000_df["취약지표"] = vulnerable_indicators
pop_10000_df.sort_values(by="취약지표", ascending=False).head()

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수,집객시설_수,인구만명당 집객 시설수,주점업 업체수,인구만명당 주점업 업체수,취약지표
20,관악구,502641,5261,104.667148,52.333574,129233,2571.079558,1074,21.367139,470,9.350610,76.749071
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440,2153,167.231094,393,30.525694,63.316737
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749,1768,116.860884,322,21.283487,58.127912
13,마포구,365612,3688,100.871963,50.435981,63172,1727.842631,1217,33.286654,659,18.024572,53.937480
4,광진구,353967,3601,101.732647,50.866324,62301,1760.079329,920,25.991123,419,11.837262,53.760012


### 경감지표
##### 인구만명당 경찰관서수(0.194), 인구만명당 범죄예방 CCTV대수(0.106)

In [16]:
cctv_seoul = pd.read_csv("data/서울CCTV현황.csv", index_col = 0)
column = ["자치구", "주소", "사용목적", "lat", "lng", "갯수", "UpdateDate"]
cctv_seoul.columns = column
cctv_seoul_total = pd.DataFrame(cctv_seoul["자치구"].value_counts())
cctv_seoul_total.reset_index(drop = False, inplace = True)
cctv_seoul_total.columns = ["자치구", "CCTV총합"]


In [17]:
pop_10000_df = pd.merge(pop_10000_df, cctv_seoul_total, how = "left", on = "자치구")
pop_10000_df["인구만명당 CCTV대수"] = pop_10000_df["CCTV총합"] / pop_10000_df["총인구(명)"] * 10000
pop_10000_df

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수,집객시설_수,인구만명당 집객 시설수,주점업 업체수,인구만명당 주점업 업체수,취약지표,CCTV총합,인구만명당 CCTV대수
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749,1768,116.860884,322,21.283487,58.127912,1649,108.995248
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440,2153,167.231094,393,30.525694,63.316737,908,70.527559
2,용산구,225882,2969,131.440310,65.720155,36881,1632.755155,982,43.474026,391,17.309923,51.591937,909,40.242250
3,성동구,291918,2362,80.913133,40.456567,42585,1458.800074,775,26.548551,269,9.214917,44.730301,1302,44.601566
4,광진구,353967,3601,101.732647,50.866324,62301,1760.079329,920,25.991123,419,11.837262,53.760012,912,25.765114
5,동대문구,351057,3401,96.878854,48.439427,60925,1735.473157,1144,32.587301,338,9.628066,53.100123,2630,74.916609
6,중랑구,385663,3726,96.612846,48.306423,56605,1467.732191,981,25.436715,379,9.827233,45.009555,3354,86.967119
7,성북구,438833,2567,58.496057,29.248028,61510,1401.672162,1083,24.679092,309,7.041403,42.716819,3478,79.255662
8,강북구,299535,2770,92.476672,46.238336,45685,1525.197389,934,31.181665,353,11.784933,47.196388,1417,47.306659
9,도봉구,315979,2179,68.960279,34.480140,34884,1103.997418,627,19.843091,220,6.962488,33.837660,1136,35.951756


In [18]:
police_substation = pd.read_csv("data/경찰청_전국 경찰관서 현황_20211231.csv", encoding="cp949")
police_substation_seoul = police_substation.loc[police_substation["전체기관명"].str.contains("서울")]
police_substation_seoul.reset_index(drop = True, inplace=True)

In [19]:
district = pop_10000_df["자치구"]
district_list = []
for gu in district:
    for police in police_substation_seoul["도로명주소"]:
        if gu in police:
            district_list.append(gu)
district_list

['종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '종로구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '중구',
 '용산구',
 '용산구',
 '용산구',
 '용산구',
 '용산구',
 '용산구',
 '용산구',
 '용산구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '성동구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '광진구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '동대문구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '중랑구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '성북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '도봉구',
 '노원구',
 '노원구',
 '노원구',
 '노원구',
 '노원구

In [20]:
from collections import Counter

counts = Counter(district_list)
police_counts = pd.DataFrame([counts.keys(), counts.values()])
police_counts = police_counts.T
police_counts.columns = ["자치구", "경찰관서수"]
police_counts

,자치구,경찰관서수
0,종로구,23
1,중구,17
2,용산구,8
3,성동구,10
4,광진구,11
5,동대문구,12
6,중랑구,9
7,성북구,12
8,강북구,10
9,도봉구,9


In [21]:
pop_10000_df = pd.merge(pop_10000_df, police_counts, how = "left", on = "자치구")
pop_10000_df

,자치구,총인구(명),5대범죄건수,인구만명당 5대범죄건수,위해지표,1인가구수,인구만명당 1인가구수,집객시설_수,인구만명당 집객 시설수,주점업 업체수,인구만명당 주점업 업체수,취약지표,CCTV총합,인구만명당 CCTV대수,경찰관서수
0,종로구,151291,3102,205.035329,102.517665,25983,1717.418749,1768,116.860884,322,21.283487,58.127912,1649,108.995248,23
1,중구,128744,3411,264.944386,132.472193,22818,1772.354440,2153,167.231094,393,30.525694,63.316737,908,70.527559,17
2,용산구,225882,2969,131.440310,65.720155,36881,1632.755155,982,43.474026,391,17.309923,51.591937,909,40.242250,8
3,성동구,291918,2362,80.913133,40.456567,42585,1458.800074,775,26.548551,269,9.214917,44.730301,1302,44.601566,10
4,광진구,353967,3601,101.732647,50.866324,62301,1760.079329,920,25.991123,419,11.837262,53.760012,912,25.765114,11
5,동대문구,351057,3401,96.878854,48.439427,60925,1735.473157,1144,32.587301,338,9.628066,53.100123,2630,74.916609,12
6,중랑구,385663,3726,96.612846,48.306423,56605,1467.732191,981,25.436715,379,9.827233,45.009555,3354,86.967119,9
7,성북구,438833,2567,58.496057,29.248028,61510,1401.672162,1083,24.679092,309,7.041403,42.716819,3478,79.255662,12
8,강북구,299535,2770,92.476672,46.238336,45685,1525.197389,934,31.181665,353,11.784933,47.196388,1417,47.306659,10
9,도봉구,315979,2179,68.960279,34.480140,34884,1103.997418,627,19.843091,220,6.962488,33.837660,1136,35.951756,9


In [22]:
pop_10000_df["인구만명당 경찰관서수"] = pop_10000_df["경찰관서수"] / pop_10000_df["총인구(명)"] * 10000
pop_10000_df["경감지표"] = (pop_10000_df["인구만명당 CCTV대수"] * 0.106) + (pop_10000_df["인구만명당 경찰관서수"] * 0.194)
pop_10000_Indicators = pop_10000_df.loc[:, ["자치구", "위해지표", "취약지표", "경감지표"]]
pop_10000_Indicators["지역안전지수"] = 100 - (pop_10000_Indicators["위해지표"] + pop_10000_Indicators["취약지표"] - pop_10000_Indicators["경감지표"])
regional_safety_index = pop_10000_Indicators.sort_values(by = "지역안전지수", ascending=False)

### 서울시 지역구 범죄 등급 지도 시각화

##### 시/군/구 5개 그룹별로 1등급 10%, 2등급 25%, 3등급 30%, 4등급25%, 5등급 10% 선정
##### 총 25개구중 1등급 3개, 2등급 6개, 3등급 8개, 4등급 6개, 5등급 2개로 구분

In [23]:
regional_safety_index.reset_index(drop=True, inplace=True)
index_grade = ["1등급", "2등급", "3등급", "4등급", "5등급"]
regional_safety_index["지수등급"] = [index_grade[0]] * 3 + [index_grade[1]] * 6 + [index_grade[2]] * 8 + [index_grade[3]] * 6 + [index_grade[4]] * 2
regional_safety_index

,자치구,위해지표,취약지표,경감지표,지역안전지수,지수등급
0,양천구,36.623029,27.872425,2.3574,37.8619,1등급
1,성북구,29.248028,42.716819,8.45415,36.4893,1등급
2,도봉구,34.480140,33.837660,3.86614,35.5483,1등급
3,노원구,36.554019,33.598603,4.19765,34.045,2등급
4,은평구,36.601008,38.427452,8.15112,33.1227,2등급
5,구로구,47.939795,36.630045,10.5782,26.0084,2등급
6,강동구,42.616001,35.294689,3.61169,25.701,2등급
7,송파구,42.049595,35.447202,1.95256,24.4558,2등급
8,성동구,40.456567,44.730301,4.79422,19.6074,2등급
9,강서구,39.080895,46.477780,4.61479,19.0561,3등급


In [24]:
import pydeck as pdk
import geopandas as gpd

geo_data = 'data/older_seoul.geojson'
df = gpd.read_file(geo_data)
df.head()

c:\Users\dlagh\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
c:\Users\dlagh\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  aout[:] = out


,시,구,동,행정동코드,인구,남자,여자,geometry
0,서울특별시,종로구,사직동,11110530,9700,4375,5325,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시,종로구,삼청동,11110540,3013,1443,1570,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시,종로구,부암동,11110550,10525,5002,5523,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시,종로구,평창동,11110560,18830,8817,10013,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시,종로구,무악동,11110570,8745,4078,4667,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."


In [25]:
geo_data1 = 'data/geodata.json'
df1 = gpd.read_file(geo_data1)
df1.head(2)

,code,name,name_eng,base_year,geometry
0,11250,강동구,Gangdong-gu,2013,"POLYGON ((127.11520 37.55753, 127.11880 37.557..."
1,11240,송파구,Songpa-gu,2013,"POLYGON ((127.06907 37.52228, 127.07496 37.520..."


In [26]:
def polygon_to_coordinates(x):
    lon, lat = x.exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

df1['coordinates'] = df1['geometry'].apply(polygon_to_coordinates)
del df1['geometry']

In [27]:
df1.rename(columns={"name": "자치구"}, inplace=True)
df1

,code,자치구,name_eng,base_year,coordinates
0,11250,강동구,Gangdong-gu,2013,"[[127.11519584981606, 37.557533180704915], [12..."
1,11240,송파구,Songpa-gu,2013,"[[127.0690698130372, 37.522279423505026], [127..."
2,11230,강남구,Gangnam-gu,2013,"[[127.05867359288398, 37.52629974922568], [127..."
3,11220,서초구,Seocho-gu,2013,"[[127.01397119667513, 37.52503988289669], [127..."
4,11210,관악구,Gwanak-gu,2013,"[[126.98367668291802, 37.473856492692086], [12..."
5,11200,동작구,Dongjak-gu,2013,"[[126.98223807916081, 37.509314966770326], [12..."
6,11190,영등포구,Yeongdeungpo-gu,2013,"[[126.89184663862764, 37.547373974997114], [12..."
7,11180,금천구,Geumcheon-gu,2013,"[[126.90156094129895, 37.47753842789901], [126..."
8,11170,구로구,Guro-gu,2013,"[[126.82688081517314, 37.50548972232896], [126..."
9,11160,강서구,Gangseo-gu,2013,"[[126.85984199399667, 37.571847855292745], [12..."


In [28]:
regional_safety_index = pd.merge(regional_safety_index, df1, how = "left", on = "자치구")
regional_safety_index

,자치구,위해지표,취약지표,경감지표,지역안전지수,지수등급,code,name_eng,base_year,coordinates
0,양천구,36.623029,27.872425,2.3574,37.8619,1등급,11150,Yangcheon-gu,2013,"[[126.8242331426722, 37.53788078753248], [126...."
1,성북구,29.248028,42.716819,8.45415,36.4893,1등급,11080,Seongbuk-gu,2013,"[[126.977175406416, 37.62859715400388], [126.9..."
2,도봉구,34.480140,33.837660,3.86614,35.5483,1등급,11100,Dobong-gu,2013,"[[127.05288479710485, 37.68423857084347], [127..."
3,노원구,36.554019,33.598603,4.19765,34.045,2등급,11110,Nowon-gu,2013,"[[127.10782277688129, 37.61804244241069], [127..."
4,은평구,36.601008,38.427452,8.15112,33.1227,2등급,11120,Eunpyeong-gu,2013,"[[126.9738864128702, 37.62949634786888], [126...."
5,구로구,47.939795,36.630045,10.5782,26.0084,2등급,11170,Guro-gu,2013,"[[126.82688081517314, 37.50548972232896], [126..."
6,강동구,42.616001,35.294689,3.61169,25.701,2등급,11250,Gangdong-gu,2013,"[[127.11519584981606, 37.557533180704915], [12..."
7,송파구,42.049595,35.447202,1.95256,24.4558,2등급,11240,Songpa-gu,2013,"[[127.0690698130372, 37.522279423505026], [127..."
8,성동구,40.456567,44.730301,4.79422,19.6074,2등급,11040,Seongdong-gu,2013,"[[127.02547266349976, 37.568943552237734], [12..."
9,강서구,39.080895,46.477780,4.61479,19.0561,3등급,11160,Gangseo-gu,2013,"[[126.85984199399667, 37.571847855292745], [12..."


In [29]:
regional_safety_index = regional_safety_index.loc[:, ["자치구", "지역안전지수", "지수등급", "coordinates"]]

In [30]:
#regional_safety_index.to_csv("regional_safety_index.csv")

In [31]:
#regional_safety_index=pd.read_csv("data/regional_safety_index.csv")

In [35]:
regional_safety_index["num"]=[100,100,100,20,20,20,20,20,20,3,3,3,3,3,3,3,3,210,210,210,210,210,210,70,70]
regional_safety_index

,자치구,지역안전지수,지수등급,coordinates,num
0,양천구,37.8619,1등급,"[[126.8242331426722, 37.53788078753248], [126....",100
1,성북구,36.4893,1등급,"[[126.977175406416, 37.62859715400388], [126.9...",100
2,도봉구,35.5483,1등급,"[[127.05288479710485, 37.68423857084347], [127...",100
3,노원구,34.045,2등급,"[[127.10782277688129, 37.61804244241069], [127...",20
4,은평구,33.1227,2등급,"[[126.9738864128702, 37.62949634786888], [126....",20
5,구로구,26.0084,2등급,"[[126.82688081517314, 37.50548972232896], [126...",20
6,강동구,25.701,2등급,"[[127.11519584981606, 37.557533180704915], [12...",20
7,송파구,24.4558,2등급,"[[127.0690698130372, 37.522279423505026], [127...",20
8,성동구,19.6074,2등급,"[[127.02547266349976, 37.568943552237734], [12...",20
9,강서구,19.0561,3등급,"[[126.85984199399667, 37.571847855292745], [12...",3


In [36]:
MAPBOX_API_KEY = "pk.eyJ1IjoiZGxhZ2h3bHMxMTA0IiwiYSI6ImNsNGRva2Z6MjBhY20zY3M2N210NGtjZmYifQ.4Lw77ZFd7aZKB3b38ymtyg"

In [43]:
# Make layer
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    regional_safety_index, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[num,num+30,num+30]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 예제 '[255*지수등급, 0, 0]'
    pickable=True, # 지도와 interactive 한 동작 on
    extruded = True,
    elevation_scale = -20,
    get_elevation = "지역안전지수",
    auto_highlight=True, # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10,
    bearing=15,
    pitch=45)

# Render
r = pdk.Deck(layers=[layer], mapbox_key = MAPBOX_API_KEY, initial_view_state=view_state)
r.to_html('a.html')